## Imports

In [44]:
import pandas as pd
import numpy as np
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from math import sqrt
from sklearn.metrics import accuracy_score
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

## Leitura e ajuste dos Datasets

In [45]:
# Definindo o caminho dos dados
data_company_path = "dados/Company.csv"
data_tweet_path = "dados/Tweet.csv"
data_company_tweet_path = "dados/Company_Tweet.csv"

In [46]:
# Ler e colocar em um dataframe
data_company = pd.read_csv(data_company_path)
data_tweet = pd.read_csv(data_tweet_path)
data_company_tweet = pd.read_csv(data_company_tweet_path)

In [47]:
print(data_company_tweet.head())
print(data_company_tweet.shape)

             tweet_id ticker_symbol
0  550803612197457920          AAPL
1  550803610825928706          AAPL
2  550803225113157632          AAPL
3  550802957370159104          AAPL
4  550802855129382912          AAPL
(4336445, 2)


In [48]:
print(data_company)
print(data_company.shape)

  ticker_symbol company_name
0          AAPL        apple
1          GOOG   Google Inc
2         GOOGL   Google Inc
3          AMZN   Amazon.com
4          TSLA    Tesla Inc
5          MSFT    Microsoft
(6, 2)


In [49]:
print(data_tweet.head())
print(data_tweet.shape)

             tweet_id           writer   post_date  \
0  550441509175443456  VisualStockRSRC  1420070457   
1  550441672312512512      KeralaGuy77  1420070496   
2  550441732014223360      DozenStocks  1420070510   
3  550442977802207232     ShowDreamCar  1420070807   
4  550443807834402816     i_Know_First  1420071005   

                                                body  comment_num  \
0  lx21 made $10,008  on $AAPL -Check it out! htt...            0   
1  Insanity of today weirdo massive selling. $aap...            0   
2  S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...            0   
3  $GM $TSLA: Volkswagen Pushes 2014 Record Recal...            0   
4  Swing Trading: Up To 8.91% Return In 14 Days h...            0   

   retweet_num  like_num  
0            0         1  
1            0         0  
2            0         0  
3            0         1  
4            0         1  
(3717964, 7)


In [50]:
dados_merge1 = pd.merge(data_company_tweet,data_company, on='ticker_symbol')
dados_merge1 = dados_merge1.drop(columns=['ticker_symbol'])
dados_merge1.head()

,tweet_id,company_name
0,550803612197457920,apple
1,550803610825928706,apple
2,550803225113157632,apple
3,550802957370159104,apple
4,550802855129382912,apple


In [51]:
dados_tweet_company = pd.merge(dados_merge1,data_tweet,on='tweet_id')
dados_tweet_company = dados_tweet_company.drop(columns=['tweet_id'])
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [52]:
#dados_tweet_company = dados_tweet_company.iloc[:500000]
#dados_tweet_company.head()

In [53]:
dados_tweet_company.shape

(4336445, 7)

## Ajustar Tempo (Data do Tweet)

In [54]:
dados_tweet_company.post_date[0]

1420156789

In [55]:
ts_epoch = dados_tweet_company.post_date[0]
ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
ts

'2015-01-01 20:59:49'

In [56]:
def ajustar_tempo(row):
    ts_epoch = row['post_date']
    ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
    return ts  
new_date = dados_tweet_company.apply(ajustar_tempo,axis=1)
dados_tweet_company['Date'] = list(new_date)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48


In [57]:
dados_tweet_company.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
company_name    object
writer          object
post_date       int64
body            object
comment_num     int64
retweet_num     int64
like_num        int64
Date            object
dtypes: int64(4), object(4)
memory usage: 297.8+ MB


## Removendo Caracteres 

In [58]:
sentenca = dados_tweet_company.body[0]
print(sentenca)

sentenca_2 = dados_tweet_company.body[100]
print(sentenca_2)

#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN $T $IGN $SPY $FB $UGAZ #sentiquant 20150101 19:00:08:298
Our Penny Stock Picks Gained Over 968% In the Past 5 Weeks! Get our next pick early: http://tinyurl.com/p46mnk2 $TRX $AAPL $GOOG


In [59]:
def replace_all(texto):
    palavras_especiais = [word for word in texto.split(' ') if word.startswith(('#','$','@'))]
    #print(palavras_especiais)
    sentenca = ' '.join(x for x in texto.split(' ') if x not in palavras_especiais)
    return sentenca

new_strings = dados_tweet_company.body.apply(replace_all)
dados_tweet_company['Processed_Tweet'] = list(new_strings)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526


In [60]:
sentenca = dados_tweet_company.Processed_Tweet[0]
print(sentenca)

sentenca_2 = dados_tweet_company.Processed_Tweet[100]
print(sentenca_2)


20150101 19:00:08:298
Our Penny Stock Picks Gained Over 968% In the Past 5 Weeks! Get our next pick early: http://tinyurl.com/p46mnk2


## Filtrando Dados

In [72]:
dados_tweet_company['Date'] = pd.to_datetime(dados_tweet_company['Date'])
dados_tweet_company1 = dados_tweet_company[dados_tweet_company['Date'].dt.year == 2015].copy()
dados_tweet_company1.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526


In [74]:
dados_tweet_company1 = dados_tweet_company1[dados_tweet_company1.company_name == 'Microsoft']
print(dados_tweet_company1.shape)
dados_tweet_company1.head()

(56971, 9)


,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet
75,Microsoft,LongbetCapital,1420145515,RT @theinformation: Apple products are getting...,0,0,2,2015-01-01 17:51:55,"RT Apple products are getting harder to use, s..."
111,Microsoft,TalkMarkets,1420139559,Gappy New Year! $SPY $DIA $IWM $INTC $MSFT $AA...,0,0,1,2015-01-01 16:12:39,Gappy New Year! http://goo.gl/RkXfHT
120,Microsoft,TalkMarkets,1420138386,3 #Technology #Stocks That Crushed #Apple In 2...,0,0,2,2015-01-01 15:53:06,3 That Crushed In 2014 http://goo.gl/t3eX4u
139,Microsoft,investcorrectly,1420133571,Key Tech Highlights Of 2014 $AAPL $BBRY $BB $M...,0,0,1,2015-01-01 14:32:51,Key Tech Highlights Of 2014 http://investcorre...
142,Microsoft,SwingTradeAlert,1420133480,IBD:2015 technology forecasts: Wearable techno...,0,0,3,2015-01-01 14:31:20,IBD:2015 technology forecasts: Wearable techno...


## Criando Labels para os Tweets 
* Positivo = 1 or pos
* Neutro = 0 or neu
* Negativo = -1 or neg

In [69]:
#'https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664'

In [76]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(dados_tweet_company1.body[142])

{'neg': 0.0, 'neu': 0.852, 'pos': 0.148, 'compound': 0.3818}

In [77]:
dados_tweet_company1.shape

(56971, 9)

In [78]:
dados_tweet_company1['vader_scores']  = None
dados_tweet_company1['vader_scores'] = dados_tweet_company1.Processed_Tweet.apply(lambda tweet : sid.polarity_scores(tweet))
dados_tweet_company1.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores
75,Microsoft,LongbetCapital,1420145515,RT @theinformation: Apple products are getting...,0,0,2,2015-01-01 17:51:55,"RT Apple products are getting harder to use, s...","{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'comp..."
111,Microsoft,TalkMarkets,1420139559,Gappy New Year! $SPY $DIA $IWM $INTC $MSFT $AA...,0,0,1,2015-01-01 16:12:39,Gappy New Year! http://goo.gl/RkXfHT,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
120,Microsoft,TalkMarkets,1420138386,3 #Technology #Stocks That Crushed #Apple In 2...,0,0,2,2015-01-01 15:53:06,3 That Crushed In 2014 http://goo.gl/t3eX4u,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp..."
139,Microsoft,investcorrectly,1420133571,Key Tech Highlights Of 2014 $AAPL $BBRY $BB $M...,0,0,1,2015-01-01 14:32:51,Key Tech Highlights Of 2014 http://investcorre...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
142,Microsoft,SwingTradeAlert,1420133480,IBD:2015 technology forecasts: Wearable techno...,0,0,3,2015-01-01 14:31:20,IBD:2015 technology forecasts: Wearable techno...,"{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'comp..."


In [79]:
# separar os valores do compound - vamos uas-lo para determinar os lbels dos tweets pos/beg
dados_tweet_company1['compound']  = dados_tweet_company1['vader_scores'].apply(lambda score_dict: score_dict['compound'])
dados_tweet_company1.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound
75,Microsoft,LongbetCapital,1420145515,RT @theinformation: Apple products are getting...,0,0,2,2015-01-01 17:51:55,"RT Apple products are getting harder to use, s...","{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'comp...",0.0258
111,Microsoft,TalkMarkets,1420139559,Gappy New Year! $SPY $DIA $IWM $INTC $MSFT $AA...,0,0,1,2015-01-01 16:12:39,Gappy New Year! http://goo.gl/RkXfHT,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
120,Microsoft,TalkMarkets,1420138386,3 #Technology #Stocks That Crushed #Apple In 2...,0,0,2,2015-01-01 15:53:06,3 That Crushed In 2014 http://goo.gl/t3eX4u,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp...",-0.4215
139,Microsoft,investcorrectly,1420133571,Key Tech Highlights Of 2014 $AAPL $BBRY $BB $M...,0,0,1,2015-01-01 14:32:51,Key Tech Highlights Of 2014 http://investcorre...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
142,Microsoft,SwingTradeAlert,1420133480,IBD:2015 technology forecasts: Wearable techno...,0,0,3,2015-01-01 14:31:20,IBD:2015 technology forecasts: Wearable techno...,"{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'comp...",0.3818


In [80]:
# definindo os labels usando o compund value como referenecia
def labels(value):
    if value == 0:
        return 'neu'
    elif value > 0:
        return 'pos'
    elif value < 0:
        return 'neg'
    
dados_tweet_company1['label']  = dados_tweet_company1['compound'].apply(lambda value: labels(value))
dados_tweet_company1.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound,label
75,Microsoft,LongbetCapital,1420145515,RT @theinformation: Apple products are getting...,0,0,2,2015-01-01 17:51:55,"RT Apple products are getting harder to use, s...","{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'comp...",0.0258,pos
111,Microsoft,TalkMarkets,1420139559,Gappy New Year! $SPY $DIA $IWM $INTC $MSFT $AA...,0,0,1,2015-01-01 16:12:39,Gappy New Year! http://goo.gl/RkXfHT,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
120,Microsoft,TalkMarkets,1420138386,3 #Technology #Stocks That Crushed #Apple In 2...,0,0,2,2015-01-01 15:53:06,3 That Crushed In 2014 http://goo.gl/t3eX4u,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp...",-0.4215,neg
139,Microsoft,investcorrectly,1420133571,Key Tech Highlights Of 2014 $AAPL $BBRY $BB $M...,0,0,1,2015-01-01 14:32:51,Key Tech Highlights Of 2014 http://investcorre...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
142,Microsoft,SwingTradeAlert,1420133480,IBD:2015 technology forecasts: Wearable techno...,0,0,3,2015-01-01 14:31:20,IBD:2015 technology forecasts: Wearable techno...,"{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'comp...",0.3818,pos


In [81]:
dados_tweet_company1.label.value_counts()

neu    28799
pos    20881
neg     7291
Name: label, dtype: int64

## Dados para ML
* Processed_Tweet - feature
* Label - target

In [82]:
# separando os valores para a machine learning
dados = dados_tweet_company1[['Processed_Tweet','label']]
dados.head()

,Processed_Tweet,label
75,"RT Apple products are getting harder to use, s...",pos
111,Gappy New Year! http://goo.gl/RkXfHT,neu
120,3 That Crushed In 2014 http://goo.gl/t3eX4u,neg
139,Key Tech Highlights Of 2014 http://investcorre...,neu
142,IBD:2015 technology forecasts: Wearable techno...,pos


## Balanceando Dados

In [83]:
dados.label.value_counts()

neu    28799
pos    20881
neg     7291
Name: label, dtype: int64

In [84]:
# filtrar dados por fase
dataset_pos = dados[dados['label']=='pos']
dataset_neu = dados[dados['label']=='neu']
dataset_neg = dados[dados['label']=='neg']

print(f'Tamanho POS: {len(dataset_pos)}')
print(f'Tamanho NEU: {len(dataset_neu)}')
print(f'Tamanho NEG: {len(dataset_neg)}')

print('#'*100)
dataset_tamanhos = [len(dataset_pos),len(dataset_neu),len(dataset_neg)]
print('Tamanhos: ',dataset_tamanhos)
print('#'*100)

tamanho_min = min(dataset_tamanhos)
print(tamanho_min)

Tamanho POS: 20881
Tamanho NEU: 28799
Tamanho NEG: 7291
####################################################################################################
Tamanhos:  [20881, 28799, 7291]
####################################################################################################
7291


In [85]:
data_a = dataset_pos.iloc[:tamanho_min-1].copy()
data_b = dataset_neu.iloc[:tamanho_min-1].copy()
data_c = dataset_neg.iloc[:tamanho_min-1].copy()

frames = [data_a, data_b,data_c]

df_data = pd.concat(frames)

In [86]:
print(df_data.shape)
df_data.head()

(21870, 2)


,Processed_Tweet,label
75,"RT Apple products are getting harder to use, s...",pos
142,IBD:2015 technology forecasts: Wearable techno...,pos
161,2015 technology forecasts: Wearable technology...,pos
171,2015 technology forecasts: Wearable technology...,pos
270,Intel: Windows 8.1 Smart TV Dongles Are Now Av...,pos


## Criando conjuntos de Treino e Teste

In [33]:
datasetVariables_train, datasetVariables_test, datasetTarget_train, datasetTarget_test = train_test_split(df_data['Processed_Tweet'], df_data['label'], test_size=0.33, random_state=42)

## Vetorizando Tweets
* Tokenizer - Tensorflow Keras

In [34]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(datasetVariables_train)

word_index = tokenizer.word_index
print(len(word_index))

21960


In [35]:
print(word_index)

{'<OOV>': 1, 'the': 2, 'to': 3, 'com': 4, 'and': 5, 'in': 6, 'a': 7, 'for': 8, 'of': 9, 'https': 10, 'is': 11, 'on': 12, 'http': 13, 'with': 14, 'this': 15, 'microsoft': 16, 'market': 17, 'stocks': 18, 'stock': 19, 'it': 20, 'at': 21, 'most': 22, '2019': 23, 'by': 24, 'up': 25, 'news': 26, '1': 27, 'as': 28, 'msft': 29, 'you': 30, 'i': 31, 'aapl': 32, 'are': 33, 'be': 34, 'today': 35, 'top': 36, 'has': 37, 'that': 38, 'not': 39, 'more': 40, 'will': 41, 'nasdaq': 42, 'apple': 43, 'inc': 44, '2': 45, 'day': 46, 'my': 47, 'from': 48, 'active': 49, 'trade': 50, 'ly': 51, 'amazon': 52, '…': 53, 'all': 54, 'out': 55, 'price': 56, '3': 57, '0': 58, 'options': 59, 'new': 60, 'its': 61, '»': 62, 'tech': 63, 'have': 64, 'now': 65, 'like': 66, '5': 67, 'but': 68, 'free': 69, 'us': 70, 'utm': 71, 'watch': 72, 'how': 73, 'week': 74, 'we': 75, '100': 76, 'if': 77, 'can': 78, 'here': 79, 'million': 80, 'buy': 81, 'amzn': 82, 't': 83, 'or': 84, 'big': 85, '10': 86, 'see': 87, 'trading': 88, 'time': 89

## Padding Vetores 
* Tamanho max dos vetores = 300

In [36]:
# Transformar as sentencas dos dados de treinamento e teste para sequencias numericas. ex [[1,2,6],[65,952,15,65],[1,2]]
train_data_sentences = tokenizer.texts_to_sequences(datasetVariables_train)
test_data_sentences = tokenizer.texts_to_sequences(datasetVariables_test)

# normalizar o tamnho das sequencias usando padding. ex [[0,1,2,6],[65,952,15,65],[0,0,1,2]]
train_padded = pad_sequences(train_data_sentences, maxlen=300)
test_padded = pad_sequences(test_data_sentences, maxlen=300)

print("Conjunto de treinamento: ","\n",train_padded[0:19])
print("Conjunto de teste: ","\n",test_padded[0:19])

Conjunto de treinamento:  
 [[   0    0    0 ...  286  737  693]
 [   0    0    0 ...    7  310    8]
 [   0    0    0 ... 1139   11   55]
 ...
 [   0    0    0 ...  399  118   74]
 [   0    0    0 ... 6059  380   55]
 [   0    0    0 ...  370  895 7424]]
Conjunto de teste:  
 [[   0    0    0 ...  340   27  133]
 [   0    0    0 ...    3   81  617]
 [   0    0    0 ...   28  227   65]
 ...
 [   0    0    0 ...  103   83  132]
 [   0    0    0 ...  148  277  153]
 [   0    0    0 ... 1428  295  491]]


In [37]:
datasetTarget_test[:15]

1688938    pos
1743931    neu
1691854    neu
1709865    pos
1764232    pos
1718046    pos
3347712    neg
1587394    neg
2249902    neg
1843612    neu
1707908    pos
1854711    neu
1849195    neg
2217359    pos
1840190    pos
Name: label, dtype: object

In [ ]:
#datasetTarget_train = to_categorical(np.array(datasetTarget_train))
#datasetTarget_test = to_categorical(np.array(datasetTarget_test))

## Machine Learning 
* Random forest

In [38]:
# first ML model
model_randomForest = RandomForestClassifier(random_state=50,criterion='gini')
model_randomForest.fit(train_padded,datasetTarget_train)

RandomForestClassifier(random_state=50)

In [39]:
predicaoTrain = model_randomForest.predict(train_padded)
predicaoTest = model_randomForest.predict(test_padded)

print("ACC TRAINING:" + str(accuracy_score(datasetTarget_train, predicaoTrain)))
print("ACC TEST:" + str(accuracy_score(datasetTarget_test, predicaoTest)))

ACC TRAINING:1.0
ACC TEST:0.6394625528738492


In [40]:
from sklearn.metrics import classification_report
print(classification_report(datasetTarget_test, predicaoTest))

              precision    recall  f1-score   support

         neg       0.67      0.85      0.75    247017
         neu       0.82      0.77      0.80    247437
         pos       0.86      0.67      0.76    247166

    accuracy                           0.77    741620
   macro avg       0.78      0.77      0.77    741620
weighted avg       0.78      0.77      0.77    741620



In [41]:
datasetTarget_test.value_counts()

neu    247437
pos    247166
neg    247017
Name: label, dtype: int64

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(datasetTarget_test, predicaoTest, labels=["neu", "pos", "neg"])

array([[191598,  12978,  42861],
       [ 18575, 166699,  61892],
       [ 23094,  14713, 209210]], dtype=int64)